In [1]:
from ast import literal_eval
import pandas as pd

In [2]:
results = (
    pd.read_csv('../datasets/gpt/tv/test.csv')
    .rename(
        columns={'explicit aspects': 'aspects'}
    )
)

gpt_results = results.columns[2:]

for col in ['aspects'] + gpt_results.tolist():
    results[col] = results[col].apply(literal_eval)

results.head()

,review,aspects,ada_results,babbage_results,curie_results,gpt_3_5_results
0,"Recomendo, uma otima smart é sansung nao preci...",[smart],[smart],[smart],[smart],"[Desculpe, mas não posso fornecer uma opinião ..."
1,"TV muito boa atendeu as expectativas , ótima p...",[tv],[tv],[tv],[tv],[tv]
2,"Bom produto, mas o som realmente deixa muito a...","[produto, som, som, base]","[produto, som, som]","[produto, som, som, som, som]","[produto, som, som]","[produto, som, som, base]"
3,Excelente produto. Estou satisfeita. Chegou an...,"[produto, serviço]","[produto, sendeçom]","[produto, serviço]","[produto, serviço]","[produto, serviço]"
4,Avalio a compra como positiva. Poderia ter um ...,"[som, imagem]","[som, imagem, brilho]","[som, imagem, brilho, tv]","[compra, som, imagem, som]","[som, imagem, imagem, recurso]"


In [3]:
# se está em GPT_RESULTS e está em ASPECTS: TP
def true_positive(y_true, y_pred):
    tp = 0
    y_true = set(y_true)
    y_pred = set(y_pred)
    for each in y_pred:
        if each in y_true:
            tp += 1
    return tp

# se está em GPT_RESULTS e NÃO está em ASPECTS: FP
def false_positive(y_true, y_pred):
    fp = 0
    y_true = set(y_true)
    y_pred = set(y_pred)
    for each in y_pred:
        if each not in y_true:
            fp += 1
    return fp

# se está em ASPECTS e NÃO está em GPT_RESULTS: FN
def false_negative(y_true, y_pred):
    fn = 0
    y_true = set(y_true)
    y_pred = set(y_pred)
    for each in y_true:
        if each not in y_pred:
            fn += 1
    return fn

In [4]:
def compute(data, by, func):
    return data.apply(lambda x: func(x['aspects'], x['curie_results']))

In [5]:
def precision(tp, fp):
    return tp / (tp + fp)

def recall(tp, fn):
    return tp / (tp + fn)

def f1_score(precision, recall):
    return 2 * (precision * recall) / (precision + recall)

In [6]:
for gpt in gpt_results:

    tp, fp, fn = 0, 0, 0
    for _, row in results.iterrows():
        y_true, y_pred = row['aspects'], row[gpt]
        tp += true_positive(y_true, y_pred)
        fp += false_positive(y_true, y_pred)
        fn += false_negative(y_true, y_pred)

    p = precision(tp, fp)
    r = recall(tp, fn)
    f1 = f1_score(p, r)

    print(f'{gpt} -> p = {p:.4f}, r = {r:.4f}, f1 = {f1:.4f}')

ada_results -> p = 0.7627, r = 0.8000, f1 = 0.7809
babbage_results -> p = 0.7787, r = 0.8133, f1 = 0.7957
curie_results -> p = 0.7530, r = 0.8400, f1 = 0.7941
gpt_3_5_results -> p = 0.7019, r = 0.6489, f1 = 0.6744
